<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
files.upload() 

In [2]:
import CreatePestImageData as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [12]:
support.Unzip('/content/LABELS_V3.0.zip','/content/Pests/Labels')

In [9]:
support.fileCount('/content/Pests/Images')

4330


In [3]:
dataSet = Imagedataset.createImageDataset()

In [4]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object ,224, '/content/Pests/Images/',  test_size=11241 )

In [ ]:
listdir('/content/Pests/Labels')

In [11]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [7]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf

Using TensorFlow backend.


In [8]:
pretrained_model = tf.keras.applications.ResNet50(input_shape=[224,224,3], include_top=False)
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

94773248/94765736 [==============================] - 1s 0us/step


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1003530   
Total params: 24,591,242
Trainable params: 1,003,530
Non-trainable params: 23,587,712
_________________________________________________________________


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

NameError: ignored

In [12]:
from tensorflow.keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [13]:
filepath="/content/drive/My Drive/Models/VGG19/weights-{epoch:02d}-{accuracy:.3f}.hdf5"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

In [14]:
tf.config.experimental_run_functions_eagerly(True)


In [16]:
history = model.fit(trainX,trainY,verbose=1,epochs=1,validation_data=(testX,testY),callbacks=[checkpoint])

1406/1406 [==============================] - ETA: 0s - loss: 5.6207 - accuracy: 0.6441
Epoch 00001: accuracy improved from -inf to 0.64407, saving model to /content/drive/My Drive/Models/VGG19/weights-01-0.644.hdf5
1406/1406 [==============================] - 4579s 3s/step - loss: 5.6207 - accuracy: 0.6441 - val_loss: 5.6033 - val_accuracy: 0.6489
